In [ ]:
import math

import numpy as np
import cv2

In [ ]:

def taylor(hm, coord):
    heatmap_width = hm.shape[0]
    px = int(coord)
    
    if 1 < px < heatmap_width-2 :

        dx  = 0.5 * (hm[px+1] - hm[px-1])
        dxx = 0.25 * (hm[px+2] - 2 * hm[px] + hm[px-2])
        derivative = np.matrix([dx])
        hessian = np.matrix([dxx])
        if dxx ** 2 != 0:
            hessianinv = hessian.I
            offset = -hessianinv * derivative
            offset = np.squeeze(np.array(offset.T), axis=0)
            coord += offset

    return coord


def gaussian_blur(hm, kernel):
    border = (kernel - 1) // 2
    # batch_size = hm.shape[0]
    num_joints = 6
    # height = hm.shape[2]
    width = hm.shape[0]
    origin_max = np.max(hm)
    dr = np.zeros((width + 2 * border))
    dr[border: -border] = hm.copy()
    kernel_1d = cv2.getGaussianKernel( 4, 2 )
    dr = cv2.filter2D(dr, -1, kernel_1d)[:,0]
    hm = dr[border: -border].copy()
    hm *= origin_max / np.max(hm)


    return hm

In [ ]:
hmap = np.asarray([0.1,0.3,0.5,0.7,0.4,0.5,0.2])  # The heatmap from each row of bottom branch representing the probabilities
coord = np.asarray([1,2,3]) # The window i want to refine i.e around start(here 2)/end with centering 
prev_coord = coord
hm = gaussian_blur(hmap, 3)
hm = np.maximum(hmap, 1e-10)
hm = np.log(hmap)
# print(hm)
new_coord = np.zeros_like(coord)
for n in range(3):
    new_coord[n] = taylor(hm, coord[n])
print("previous window",prev_coord)
print("refined window",new_coord) # The window i got after refinement
print("previous heatmap", hmap)
for i in range(3):
    hmap[prev_coord[i]] = hmap[new_coord[i]]
print("current heatmap",hmap)

previous window [1 2 3]
refined window [1 2 2]
previous heatmap [0.1 0.3 0.5 0.7 0.4 0.5 0.2]
current heatmap [0.1 0.3 0.5 0.5 0.4 0.5 0.2]
